# Задание в классе

В этом задание будем работать с данными от Samsung: `Human Activity Recognition`. Скачайте датасет [тут](https://drive.google.com/file/d/14RukQ0ylM2GCdViUHBBjZ2imCaYcjlux/view?usp=sharing). Данные собранны с Samsung Galaxy S3 и размечены, каждая активность соответсвует ходьбе (walking), стоянию (standing), сидению (sitting), лежанию (laying down) и ходьбе вверх и вниз по лестнице (walking upstairs/downstairs).

Напишите код, где требуется, ответьте на вопросы и заполните [форму](https://forms.gle/S82a8xr8kbPYg46x7).


In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from matplotlib import pyplot as plt
plt.style.use(['seaborn-darkgrid'])
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.family'] = 'DejaVu Sans'

from sklearn import metrics
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

RANDOM_STATE = 7
PATH_TO_SAMSUNG_DATA = "./data/samsung_HAR/"

%matplotlib inline

In [ ]:
X_train = np.loadtxt(os.path.join(PATH_TO_SAMSUNG_DATA, "samsung_train.txt"))
y_train = np.loadtxt(os.path.join(PATH_TO_SAMSUNG_DATA, "samsung_train_labels.txt")).astype(int)

X_test = np.loadtxt(os.path.join(PATH_TO_SAMSUNG_DATA, "samsung_test.txt"))
y_test = np.loadtxt(os.path.join(PATH_TO_SAMSUNG_DATA, "samsung_test_labels.txt")).astype(int)

labels_mapping = {
    1: 'walking',
    2: 'walking upstairs',
    3: 'walking downstairs',
    4: 'sitting',
    5: 'standing',
    6: 'laying down'
}

assert(X_train.shape == (7352, 561) and y_train.shape == (7352,))
assert(X_test.shape == (2947, 561) and y_test.shape == (2947,))

for i in np.random.choice(X_train.shape[0], 10):
    print(f"{labels_mapping[y_train[i]]}: {', '.join(map(str, X_train[i, :5]))}...")

Для задачи кластеризации нам не нужно разделение на тренировочную и тестовую выборку, поэтому объедените массивы.

In [ ]:
X = None
y = None

Стандартизируйте значения, использя `StandartScaler` с параметрами по умолчанию

In [ ]:
X_scaled = None

Уменьшите количество параметров с помощью `PCA`.

**Вопрос 1.** Какое наименьшее число компонет покрывает 90% дисперсии стандартизованных данных?

In [ ]:
pca = None
X_pca = None

**Вопрос 2.** Какой процент дисперсии объясняют первые 5 компонент?
*округлите до целого числа*

In [ ]:
round(... * 100)

Визуализируйте данные, используя первые 2 компоненты. Используйте `plt.scatter` с указанием метки добавляемого класса. 

**Вопрос 3.** Сколько кластеров явно выделилось? Каким активностям они соответствуют?

В качестве ответа укажите количество кластеров, а так как же какие активности принадлежат каждому кластеру в свободной форме.

In [ ]:
plt.scatter(...)
plt.show()

Обучите модель `K-means`, используя сжатые наблюдения (`X_pca`), параметры:
- n_clusters = number of clusters
- n_init = 100
- random_state = RANDOM_STATE

In [ ]:
kmeans = None

Визуализируйте данные, используя первые 2 компоненты, но теперь окрасьте точки в соответствии с построенным разбиением.

In [ ]:
plt.scatter(...)
plt.show()

Посмотрим на соответствие между метками и построенными кластерами

In [ ]:
tab = pd.crosstab(y, kmeans.labels_, margins=True)
tab.index = list(labels_mapping.values()) + ['all']
tab.columns = [f'cluster_{i+1}' for i in range(6)] + ['all']
tab

Можно увидеть, что каждый класс попал в несколько кластеров. Давайте рассмотрим максимальный процент объектов в классе, которые попали в один кластер. Например, если в классе 1406 примеров и разбиение на классы:
- 900
- 500
- 6  
то значений такой метрики будет $\frac{900}{1406} \approx 0.64$

Такая простая метрика показывает, насколько класс отделим от других.

Посчитайте такую метрику для каждого класса.

**Вопрос 4.** Отсортируйте класс в порядке уменьшения метрики. В качестве ответа, укажите номера классов через пробел

1 - walking, 2 - walking upstairs, 3 - walking downstairs, 4 - sitting, 5 - standing, 6 - laying down

Возможно, что 6 $-$ не лучшее число кластеров. Определите с помощью метода "локтя", оптимальное число кластеров.

**Вопрос 5.** Какое число кластеров оптимально?

In [ ]:
inertia = []
for k in tqdm_notebook(range(1, 7)):
    pass

In [ ]:
plt.plot(range(1, 7), inertia, marker='s');

Давайте попробуем другой тип кластеризации $-$ агломеративную кластеризацию.

Обучите её на сжатых данных, в качестве количества кластеров укажите число классов, а параметр `linkage` $-$ `ward`

In [ ]:
ag = None

Сравните полученную модель с `K-means` из 4 вопроса с помощью метрики $ARI$

In [ ]:
print('KMeans: ARI =', None)
print('Agglomerative CLustering: ARI =', None)

**Вопрос 6.** Выберите верные утверждения

- Согласно $ARI$, агломеративная кластеризация работает лучше, чем k-means;
- Если перенумеровать классы, то значение метрики не изменится;
- Для случайного разделения на классы, метрика будет близка к 0;
- Метрику $ARI$ можно использовать, даже когда нет истинных меток класса.

Для определения типа активности можно решать и задачу мультиклассовой классификации, так как уже известны метки классов. Воспользуемся моделью `LinearSVC`, а для подбора оптимальных параметров будем использовать поиск по сетке и кросс-валидацию. Почитать про это можно [тут](https://towardsdatascience.com/why-and-how-to-cross-validate-a-model-d6424b45261f).

- Используйте `StandartScaler` для стандартизирования данных, но обучите его только на тренировочных данных
- Подберите параметр `C` с помощью `GridSearchCV`, параметр `cv` установите 3

In [ ]:
svc = LinearSVC(random_state=RANDOM_STATE)
svc_params = {'C': [0.001, 0.01, 0.1, 1, 10]}

In [ ]:
X_train_scaled = None
X_test_scaled = None

In [ ]:
best_svc = GridSearchCV(...)
best_svc.fit(X_train_scaled, y_train)

In [ ]:
best_svc.best_params_, best_svc.best_score_

**Вопрос 7.** Какой оптимальный параметр?

In [ ]:
y_predicted = best_svc.predict(X_test_scaled)

In [ ]:
tab = pd.crosstab(y_test, y_predicted, margins=True)
tab.index = list(labels_mapping.values()) + ['all']
tab.columns = list(labels_mapping.values()) + ['all']
tab

Посчитайте снова метрику отделимости каждого класса.

**Вопрос 8.** Линейный классификатор справился лучше, чем k-means?

**Вопрос 9.** Какая активность хуже всего распознаётся линейным классификатором?

**Вопрос 10.** Выберите верные утверждения

- PCA позволяет ускорить время обучения, но качество, как правило, становится хуже;
- С помощью PCA можно визуализировать данные на плоскости, но это не единственный способ, например, можно использовать t-SNE;
- Каждая компонента PCA $-$ это линейная комбинация исходных параметров.